In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp
import numpy as np

In [2]:
torch.cuda.device_count()

1

In [3]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst
def dumppkl(data,path):
    with open(path,'wb') as f:
        pickle.dump(data,f)

In [4]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_id_abs_title_cite_ref_author.pkl"
full_data=loadpkl(path)

In [5]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_train_pred_1000.pkl"
train_top_1000=loadpkl(path)

In [6]:
pos_pair=[]
neg_pair=[]
for k,v in train_top_1000.items():
    if k not in full_data.keys():
        continue
    for paper in v[:500]:
        if paper not in full_data.keys():
            continue
        if paper in full_data[k]["references"]:
            pos_pair.append((k,paper))
        else:
            neg_pair.append((k,paper))
            

In [7]:
print(len(pos_pair))
print(len(neg_pair))

95099
4265401


In [8]:
neg_pair=list(set(neg_pair))

In [9]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/train_cite_pair.pkl"
train_cite_norm=loadpkl(path)
print(len(train_cite_norm))

5288766


In [10]:
pair_training=[]
for pair in neg_pair:
    data={}
    data['query']=pair[0]
    data['key']=pair[1]
    data['label']=0
    pair_training.append(data)
for pair in train_cite_norm:
    data={}
    data['query']=pair[0]
    data['key']=pair[1]
    data['label']=1
    pair_training.append(data)


In [11]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/train_cite_pair_label.pkl"
dumppkl(pair_training,path)

In [12]:
print(len(pair_training))

9554167


In [13]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_dev_pred_1000.pkl"
dev_pred_1000=loadpkl(path)

In [14]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_dev_gold.pkl"
dev_gold=loadpkl(path)

In [15]:
pos_pair=[]
for k,v in dev_gold.items():
    if k not in full_data.keys():
        continue
    for paper in v:
        if paper not in full_data.keys():
            continue
        if paper in full_data[k]["references"]:
            pos_pair.append((k,paper))
       
            

In [16]:
neg_pair=[]
for k,v in dev_pred_1000.items():
    if k not in full_data.keys():
        continue
    for paper in v[:500]:
        if paper not in full_data.keys():
            continue
        if paper in full_data[k]["references"]:
            continue
        else:
            neg_pair.append((k,paper))
            

In [17]:
print(len(pos_pair))
print(len(neg_pair))

35507
429734


In [18]:
pair_dev=[]
for pair in neg_pair:
    data={}
    data['query']=pair[0]
    data['key']=pair[1]
    data['label']=0
    pair_dev.append(data)
for pair in pos_pair:
    data={}
    data['query']=pair[0]
    data['key']=pair[1]
    data['label']=1
    pair_dev.append(data)

In [19]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/dev_cite_pair_label.pkl"
dumppkl(pair_dev,path)

In [21]:
print(len(pair_dev))
print(len(pair_training))

465241
9554167
